# PAIN ASSESSMENT - EXPERIMENTS

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.initializers import HeNormal
from scipy import stats
from scipy.stats import f_oneway
from scipy.stats import kruskal
import statsmodels.api as sm

In [2]:
df_all_participants = pd.read_excel('experiment_11_02_2023_Catch22_Normalized.xlsx')
df_all_participants

,EDA_DN_HistogramMode_5,EDA_DN_HistogramMode_10,EDA_CO_f1ecac,EDA_CO_FirstMin_ac,EDA_CO_HistogramAMI_even_2_5,EDA_CO_trev_1_num,EDA_MD_hrv_classic_pnn40,EDA_SB_BinaryStats_mean_longstretch1,EDA_SB_TransitionMatrix_3ac_sumdiagcov,EDA_PD_PeriodicityWang_th0_01,...,EMG_DN_OutlierInclude_p_001_mdrmd,EMG_DN_OutlierInclude_n_001_mdrmd,EMG_SP_Summaries_welch_rect_area_5_1,EMG_SB_BinaryStats_diff_longstretch0,EMG_SB_MotifThree_quantile_hh,EMG_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,EMG_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,EMG_SP_Summaries_welch_rect_centroid,EMG_FC_LocalSimple_mean3_stderr,pain_type
0,-0.612005,-0.209698,0.643970,0.404148,0.665238,0.353551,-0.353553,-0.068585,-0.972239,-0.705932,...,-1.106187,-1.597055,0.283918,0.101015,-0.714623,-0.390434,-1.201852e-15,0.225308,-0.036091,baseline
1,-1.115335,0.420989,0.705956,0.904075,0.851948,0.353554,-0.353553,0.224589,0.362493,-0.705932,...,-0.861846,-0.637219,-0.202222,1.010153,-0.299962,0.312348,8.660254e-01,0.974483,0.607675,l_cold
2,0.771177,-0.995415,0.885025,0.683850,0.789856,0.353554,-0.353553,0.096949,0.362493,1.490503,...,0.267412,0.717544,-0.293116,-0.808122,-0.616589,1.717911,8.660254e-01,0.155385,0.302297,h_cold
3,1.840184,2.034122,0.427183,0.436700,0.339704,0.353554,-0.353553,0.333782,0.362493,-0.705932,...,-0.512571,-0.318504,-1.378682,-1.717259,1.046525,1.717911,8.660254e-01,0.974483,1.118428,l_heat_l_cold
4,0.186699,-0.486304,-1.062932,-1.175204,-1.133315,0.353553,-0.353553,-1.195908,-1.372658,1.474063,...,-0.133862,-0.199996,-1.760514,-0.808122,1.788780,-1.093216,8.660254e-01,1.204230,1.381565,h_heat_l_cold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,-1.067311,-1.564824,0.902719,0.874688,0.979679,0.438197,0.000000,0.018925,0.856640,0.000000,...,0.031606,-0.157918,0.345431,-0.150756,0.016306,0.101015,0.000000e+00,-0.279397,-0.111078,h_heat_l_cold
275,0.253522,-0.265192,0.629750,1.100550,0.910354,-0.078712,0.000000,-0.579999,0.856640,0.000000,...,0.649127,0.637840,-1.363417,-0.150756,1.092183,0.101015,0.000000e+00,1.349538,1.184624,l_heat_h_cold
276,-1.134167,0.785141,0.908561,0.854707,1.021532,0.402872,0.000000,0.010548,0.856640,0.000000,...,0.655000,1.117939,-1.635537,-1.507557,1.163303,1.010153,0.000000e+00,1.570947,1.243630,h_heat_h_cold
277,0.953090,0.762769,-0.959148,-1.029510,-1.589207,0.343430,0.000000,0.948723,-1.145895,0.000000,...,1.234765,1.225362,1.831392,2.110579,-2.345720,0.101015,0.000000e+00,-1.671108,-2.210266,l_heat


In [4]:
df_all_participants_filtered = df_all_participants.loc[:, df_all_participants.apply(pd.Series.nunique) != 1]
df_all_participants_filtered = df_all_participants_filtered.dropna(axis=1)

print("Original DataFrame shape:", df_all_participants.shape)
print("Filtered DataFrame shape:", df_all_participants_filtered.shape)

# Find columns with infinity values
columns_with_infinity = df_all_participants_filtered.columns[df_all_participants_filtered.isin([np.inf, -np.inf]).any()]

# Remove columns with infinity values
df_all_participants_filtered = df_all_participants_filtered.drop(columns=columns_with_infinity)
print("Filtered DataFrame shape:", df_all_participants_filtered.shape)

df_all_participants = df_all_participants_filtered
print("Filtered DataFrame shape:", df_all_participants.shape)

Original DataFrame shape: (279, 89)
Filtered DataFrame shape: (279, 87)
Filtered DataFrame shape: (279, 87)
Filtered DataFrame shape: (279, 87)


In [6]:
def filter_data(df,label1, label2, label3=None):
    if label3 is None:
        filter_list = [label1, label2]
    else:
        filter_list = [label1, label2, label3]
    
    df = df[df['pain_type'].isin(filter_list)]
    print(df.pain_type.unique())
    return df

def replace_name(df, replacement_dict):
    df = df.copy()
    df.loc[:, 'pain_type'] = df['pain_type'].replace(replacement_dict)
    print(df.pain_type.unique())
    pain_type_counts = df['pain_type'].value_counts()
    # print(pain_type_counts)

    return df

################################################
### Chose which levels of pain to compare ###
################################################

# filter_list = ['h_cold', 'l_heat_h_cold','h_heat_h_cold']
# filter_list = ['l_cold', 'l_heat_l_cold','h_heat_l_cold']


# # high cold
# filter_list = ['h_cold', 'l_heat_h_cold']
# filter_list = ['h_cold', 'h_heat_h_cold']
filter_list = ['l_heat_h_cold', 'h_heat_h_cold']
# # low cold
# filter_list = ['l_cold', 'l_heat_l_cold']
# filter_list = ['l_cold', 'h_heat_l_cold']
# filter_list = ['l_heat_l_cold', 'h_heat_l_cold']

df_to_run = filter_data(df_all_participants, filter_list[0],filter_list[1])
df_to_run.shape

['l_heat_h_cold' 'h_heat_h_cold']


(62, 87)

In [8]:
replacement_dict = {'h_heat': 'high_heat', 'l_heat': 'low_heat', 
                    'h_cold': 'high_cold', 'l_cold': 'low_cold',
                   'h_heat_h_cold': 'high_cold_severe_existing_pain',
                   'l_heat_l_cold': 'low_cold_mild_existing_pain',
                   'l_heat_h_cold': 'high_cold_mild_existing_pain',
                   'h_heat_l_cold': 'low_cold_severe_existing_pain',
                    'baseline':'no_pain'
                   }

df_to_run = replace_name(df_to_run, replacement_dict)
print(df_to_run.shape)
top_features = df_to_run.columns.tolist()
len(top_features)

['high_cold_mild_existing_pain' 'high_cold_severe_existing_pain']
(62, 87)


87

In [76]:
df_last = df_to_run
df_last

,EDA_DN_HistogramMode_5,EDA_DN_HistogramMode_10,EDA_CO_f1ecac,EDA_CO_FirstMin_ac,EDA_CO_HistogramAMI_even_2_5,EDA_CO_trev_1_num,EDA_MD_hrv_classic_pnn40,EDA_SB_BinaryStats_mean_longstretch1,EDA_SB_TransitionMatrix_3ac_sumdiagcov,EDA_PD_PeriodicityWang_th0_01,...,EMG_DN_OutlierInclude_p_001_mdrmd,EMG_DN_OutlierInclude_n_001_mdrmd,EMG_SP_Summaries_welch_rect_area_5_1,EMG_SB_BinaryStats_diff_longstretch0,EMG_SB_MotifThree_quantile_hh,EMG_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,EMG_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,EMG_SP_Summaries_welch_rect_centroid,EMG_FC_LocalSimple_mean3_stderr,pain_type
5,1.012252,1.158563,-0.863328,-1.122961,-1.048948,0.353552,-0.353553,2.021027,-1.372658,1.271024,...,-0.626379,-0.618366,0.308025,0.101015,-0.756059,-0.390434,-1.201852e-15,0.355165,0.193531,high_cold_mild_existing_pain
6,-1.147762,-0.648143,-2.058054,-1.803730,-1.909591,-2.828427,2.828427,-1.788239,1.905089,-0.705932,...,-0.252576,-0.250272,0.781177,1.919290,-0.648372,-0.390434,-1.201852e-15,-0.973373,-0.883299,high_cold_severe_existing_pain
14,-1.265705,-1.250014,0.718868,0.563235,0.714439,0.192401,0.000000,-0.368479,0.806090,0.000000,...,2.024632,1.076852,-2.001253,-2.351802,2.077615,-1.206045,0.000000e+00,1.995403,2.069797,high_cold_mild_existing_pain
15,-0.003825,0.141887,0.291642,1.119948,1.016533,-2.565536,0.000000,0.903842,0.806090,0.000000,...,1.118501,1.610725,-1.352132,-0.658505,0.981666,0.150756,0.000000e+00,1.058431,1.073753,high_cold_severe_existing_pain
23,-0.708192,-0.882371,0.741839,0.427322,0.578919,0.200953,0.000000,0.006881,0.533337,0.000000,...,0.182892,0.444291,1.383594,-0.408248,-0.975154,-1.897367,1.206045e+00,-0.944557,-0.951603,high_cold_mild_existing_pain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,-1.304708,-1.078440,1.100459,0.765483,0.510925,-0.259391,0.000000,0.220045,0.892462,0.000000,...,-0.242929,-0.281275,-0.001343,-0.515711,0.199762,-1.900658,1.118034e+00,0.751322,-0.162829,high_cold_severe_existing_pain
266,-0.834957,-2.321185,-1.037869,1.098137,0.259725,1.325646,0.000000,-1.282622,-0.619288,0.000000,...,-0.805812,-0.676504,-1.965595,-0.288675,1.385779,1.010153,0.000000e+00,-0.076696,1.077523,high_cold_mild_existing_pain
267,0.853989,0.800628,-0.198132,-0.535445,0.747361,0.711741,0.000000,0.059776,-0.619288,0.000000,...,2.183439,1.523682,-1.280369,-2.020726,1.469279,0.101015,0.000000e+00,1.703470,1.896382,high_cold_severe_existing_pain
275,0.253522,-0.265192,0.629750,1.100550,0.910354,-0.078712,0.000000,-0.579999,0.856640,0.000000,...,0.649127,0.637840,-1.363417,-0.150756,1.092183,0.101015,0.000000e+00,1.349538,1.184624,high_cold_mild_existing_pain


In [77]:
print(df_last.shape)
# Separate the target variable from the features
target = df_last['pain_type']
features = df_last.drop(columns='pain_type')

# Define a correlation threshold (e.g., 0.85 for highly correlated)
correlation_threshold = 0.9999

while True:
    # Calculate the correlation matrix for the features
    correlation_matrix = features.corr()
    
    # Create a mask for highly correlated variables
    mask = (correlation_matrix.abs() >= correlation_threshold)
    
    # Identify and count highly correlated variables for each feature
    correlated_counts = mask.sum()
    
    # Sort features by the number of correlated variables in descending order
    features_sorted = correlated_counts.sort_values(ascending=False)
    
    # Get the most highly correlated feature
    most_correlated_feature = features_sorted.index[0]
    
    # Remove the most highly correlated feature
    features = features.drop(columns=most_correlated_feature)
    
    # Check if all remaining variables are correlated less than the threshold
    if (mask.sum() <= 1).all():
        break

# Combine the features and target variable into a new DataFrame
df = pd.concat([features, target], axis=1)

# Print the updated DataFrame
df_last = df
print(df_last.shape)


(62, 87)
(62, 86)


In [78]:
top_features= df_last.columns.tolist()
print(len(top_features))
top_features.remove('pain_type')
print(len(top_features))

print(df_last.pain_type.unique())

86
85
['high_cold_mild_existing_pain' 'high_cold_severe_existing_pain']


In [79]:
df_last

,EDA_DN_HistogramMode_10,EDA_CO_f1ecac,EDA_CO_FirstMin_ac,EDA_CO_HistogramAMI_even_2_5,EDA_CO_trev_1_num,EDA_MD_hrv_classic_pnn40,EDA_SB_BinaryStats_mean_longstretch1,EDA_SB_TransitionMatrix_3ac_sumdiagcov,EDA_PD_PeriodicityWang_th0_01,EDA_CO_Embed2_Dist_tau_d_expfit_meandiff,...,EMG_DN_OutlierInclude_p_001_mdrmd,EMG_DN_OutlierInclude_n_001_mdrmd,EMG_SP_Summaries_welch_rect_area_5_1,EMG_SB_BinaryStats_diff_longstretch0,EMG_SB_MotifThree_quantile_hh,EMG_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,EMG_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,EMG_SP_Summaries_welch_rect_centroid,EMG_FC_LocalSimple_mean3_stderr,pain_type
5,1.158563,-0.863328,-1.122961,-1.048948,0.353552,-0.353553,2.021027,-1.372658,1.271024,1.649319,...,-0.626379,-0.618366,0.308025,0.101015,-0.756059,-0.390434,-1.201852e-15,0.355165,0.193531,high_cold_mild_existing_pain
6,-0.648143,-2.058054,-1.803730,-1.909591,-2.828427,2.828427,-1.788239,1.905089,-0.705932,-0.693136,...,-0.252576,-0.250272,0.781177,1.919290,-0.648372,-0.390434,-1.201852e-15,-0.973373,-0.883299,high_cold_severe_existing_pain
14,-1.250014,0.718868,0.563235,0.714439,0.192401,0.000000,-0.368479,0.806090,0.000000,-0.520134,...,2.024632,1.076852,-2.001253,-2.351802,2.077615,-1.206045,0.000000e+00,1.995403,2.069797,high_cold_mild_existing_pain
15,0.141887,0.291642,1.119948,1.016533,-2.565536,0.000000,0.903842,0.806090,0.000000,1.331595,...,1.118501,1.610725,-1.352132,-0.658505,0.981666,0.150756,0.000000e+00,1.058431,1.073753,high_cold_severe_existing_pain
23,-0.882371,0.741839,0.427322,0.578919,0.200953,0.000000,0.006881,0.533337,0.000000,0.000000,...,0.182892,0.444291,1.383594,-0.408248,-0.975154,-1.897367,1.206045e+00,-0.944557,-0.951603,high_cold_mild_existing_pain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,-1.078440,1.100459,0.765483,0.510925,-0.259391,0.000000,0.220045,0.892462,0.000000,0.000000,...,-0.242929,-0.281275,-0.001343,-0.515711,0.199762,-1.900658,1.118034e+00,0.751322,-0.162829,high_cold_severe_existing_pain
266,-2.321185,-1.037869,1.098137,0.259725,1.325646,0.000000,-1.282622,-0.619288,0.000000,-0.353553,...,-0.805812,-0.676504,-1.965595,-0.288675,1.385779,1.010153,0.000000e+00,-0.076696,1.077523,high_cold_mild_existing_pain
267,0.800628,-0.198132,-0.535445,0.747361,0.711741,0.000000,0.059776,-0.619288,0.000000,-0.353553,...,2.183439,1.523682,-1.280369,-2.020726,1.469279,0.101015,0.000000e+00,1.703470,1.896382,high_cold_severe_existing_pain
275,-0.265192,0.629750,1.100550,0.910354,-0.078712,0.000000,-0.579999,0.856640,0.000000,0.934838,...,0.649127,0.637840,-1.363417,-0.150756,1.092183,0.101015,0.000000e+00,1.349538,1.184624,high_cold_mild_existing_pain


# Anova

In [80]:
# ANOVA
alpha = 0.05  # Set your desired significance level

# Create an empty DataFrame
important_features_df = pd.DataFrame(columns=["Feature 1",  "p_value"])
important_features_dfs = []
significant_features_list = []

for which_features in range(0,len(top_features)):
    grouped_data = [df_last[df_last['pain_type'] == category][top_features[which_features]] for category in df_last['pain_type'].unique()]
    # print(df_last['pain_type'].unique())

    # Perform the ANOVA test
    f_statistic, p_value = f_oneway(*grouped_data)
    # # Perform the Kruskal-Wallis H test
    # h_statistic, p_value_kruskal = kruskal(*grouped_data)

    
    if p_value <= 0.05:
        # Interpret the results
        # print("F-Statistic:", f_statistic)
        print(top_features[which_features])
        print("P-Value:", p_value)
    
        significant_features_list = [top_features[which_features], p_value]

        print(which_features)

#         # Create a DataFrame for the significant features and append it to the list
        important_features_dfs.append(pd.DataFrame([significant_features_list], columns=["Feature 1", "p_value"]))
#         # if p_value < alpha:
#         #     print("Reject the null hypothesis: There is a statistically significant difference between the groups.")
#         # else:
#         #     print("Fail to reject the null hypothesis: There is no statistically significant difference between the groups.")

# Concatenate the list of DataFrames into a single DataFrame
important_features_df = pd.concat(important_features_dfs, ignore_index=True)

important_features_df = important_features_df.sort_values(by='p_value', ascending=True)
important_features_df


EDA_SB_TransitionMatrix_3ac_sumdiagcov
P-Value: 0.016416861183777175
7
EDA_DN_OutlierInclude_n_001_mdrmd
P-Value: 0.01109879601259657
13
EMG_rmseDN_OutlierInclude_p_001_mdrmd
P-Value: 0.009366454054628548
33
EMG_rmseDN_OutlierInclude_n_001_mdrmd
P-Value: 0.0051132916759912906
34
EMG_rec_FC_LocalSimple_mean1_tauresrat
P-Value: 0.023967950279558777
53
EMG_rec_DN_OutlierInclude_p_001_mdrmd
P-Value: 0.010661175903558334
54
EMG_rec_DN_OutlierInclude_n_001_mdrmd
P-Value: 0.009938526950006455
55
EMG_DN_HistogramMode_10
P-Value: 0.030309336080277624
64
EMG_DN_OutlierInclude_n_001_mdrmd
P-Value: 0.04002763157555641
77


,Feature 1,p_value
3,EMG_rmseDN_OutlierInclude_n_001_mdrmd,0.005113
2,EMG_rmseDN_OutlierInclude_p_001_mdrmd,0.009366
6,EMG_rec_DN_OutlierInclude_n_001_mdrmd,0.009939
5,EMG_rec_DN_OutlierInclude_p_001_mdrmd,0.010661
1,EDA_DN_OutlierInclude_n_001_mdrmd,0.011099
0,EDA_SB_TransitionMatrix_3ac_sumdiagcov,0.016417
4,EMG_rec_FC_LocalSimple_mean1_tauresrat,0.023968
7,EMG_DN_HistogramMode_10,0.030309
8,EMG_DN_OutlierInclude_n_001_mdrmd,0.040028


In [81]:
important_features_df_1feature = important_features_df
important_features_1feature_list = list(important_features_df_1feature['Feature 1'].unique())
important_features_1feature_list

['EMG_rmseDN_OutlierInclude_n_001_mdrmd',
 'EMG_rmseDN_OutlierInclude_p_001_mdrmd',
 'EMG_rec_DN_OutlierInclude_n_001_mdrmd',
 'EMG_rec_DN_OutlierInclude_p_001_mdrmd',
 'EDA_DN_OutlierInclude_n_001_mdrmd',
 'EDA_SB_TransitionMatrix_3ac_sumdiagcov',
 'EMG_rec_FC_LocalSimple_mean1_tauresrat',
 'EMG_DN_HistogramMode_10',
 'EMG_DN_OutlierInclude_n_001_mdrmd']